In [23]:
import numpy as numpy
import pandas as pd
import matplotlib.pyplot as pit
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.metrics import classification_report,confusion_matrix
import missingno as mns
import nltk
from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
nltk.download('stopwords')
stop_words = stopwords.words('english')
from textblob import TextBlob
from sklearn.pipeline import Pipeline

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [24]:
df = pd.read_csv('all_combined.csv')
df.head()

,reviewId,content,score,app
0,5fed15ee-a43a-46a2-ba64-49c63679a022,osm,5,Facebook
1,4a98fc8c-8d72-45d5-b845-413bbc6a6ecf,✋🏿😔🤚🏿✋🏿😔🤚🏿 PURIHIN ANG PANGINOON,5,Facebook
2,597b0f7d-90c6-4ca9-979e-b7d20d2a780c,Sushnti kumari,1,Facebook
3,9f4a526b-b9f6-4743-9714-4bb07be6fcb2,Very nice,5,Facebook
4,b847e6e2-f9c5-444b-b4d6-8d8f7b72a1c1,I've reported an obvious fake profile with zer...,1,Facebook


In [25]:
df.isnull().sum()

reviewId     0
content     37
score        0
app          0
dtype: int64

In [26]:
df.dropna(inplace=True)

In [27]:
def getSubjectivity(text):
    if isinstance(text,str):
        return TextBlob(text).sentiment.subjectivity
    else:
        return None
def getpolarity(text):
    if isinstance(text,str):
        return TextBlob(text).sentiment.polarity
    else:
        return None
df['subjectivity'] = df['content'].apply(getSubjectivity)
df['polarity'] = df['content'].apply(getpolarity)
df.head()

,reviewId,content,score,app,subjectivity,polarity
0,5fed15ee-a43a-46a2-ba64-49c63679a022,osm,5,Facebook,0.000000,0.00000
1,4a98fc8c-8d72-45d5-b845-413bbc6a6ecf,✋🏿😔🤚🏿✋🏿😔🤚🏿 PURIHIN ANG PANGINOON,5,Facebook,0.000000,0.00000
2,597b0f7d-90c6-4ca9-979e-b7d20d2a780c,Sushnti kumari,1,Facebook,0.000000,0.00000
3,9f4a526b-b9f6-4743-9714-4bb07be6fcb2,Very nice,5,Facebook,1.000000,0.78000
4,b847e6e2-f9c5-444b-b4d6-8d8f7b72a1c1,I've reported an obvious fake profile with zer...,1,Facebook,0.507937,0.05754


In [28]:
def outcome(score):
    if score < 0:
        return 'negative'
    elif score == 0:
        return 'neutral'
    else:
        return 'positive'
df['Outcome'] = df['polarity'].apply(outcome)
df

,reviewId,content,score,app,subjectivity,polarity,Outcome
0,5fed15ee-a43a-46a2-ba64-49c63679a022,osm,5,Facebook,0.000000,0.00000,neutral
1,4a98fc8c-8d72-45d5-b845-413bbc6a6ecf,✋🏿😔🤚🏿✋🏿😔🤚🏿 PURIHIN ANG PANGINOON,5,Facebook,0.000000,0.00000,neutral
2,597b0f7d-90c6-4ca9-979e-b7d20d2a780c,Sushnti kumari,1,Facebook,0.000000,0.00000,neutral
3,9f4a526b-b9f6-4743-9714-4bb07be6fcb2,Very nice,5,Facebook,1.000000,0.78000,positive
4,b847e6e2-f9c5-444b-b4d6-8d8f7b72a1c1,I've reported an obvious fake profile with zer...,1,Facebook,0.507937,0.05754,positive
...,...,...,...,...,...,...,...
199995,79dde4cf-cd59-4931-be2e-306943eb2565,remove expired from photos & videos pls :)). i...,3,LINE,1.000000,0.00000,neutral
199996,72cde2ba-9f80-4547-bcb3-f63ce89548c1,Nice,5,LINE,1.000000,0.60000,positive
199997,83c741f2-9281-4eec-84a8-f749e382bbb1,It's good,1,LINE,0.600000,0.70000,positive
199998,e11cd2f8-2a31-465e-bac5-df708e026082,A great app but sadly we can't delete our mess...,3,LINE,0.916667,-0.15000,negative


In [29]:
text_cleaning_re = "@\S+|https?:\S+http?:\s|[^A-Za-z0-9]+"
def preprocess(x, stem = False):
    x = re.sub(text_cleaning_re, " ",str(x).lower()).strip()
    tokens = []
    for token in x.split():
            if token not in stop_words:
              
                 if stem:
                     tokens.append(SnowballStemmer.stem(token))
                 else:
                      tokens.append(token)
    return " ".join(tokens)
df['content'] = df['content'].apply(lambda x: preprocess(x))


<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\hp\AppData\Local\Temp\ipykernel_15532\2854997355.py:1: SyntaxWarning: invalid escape sequence '\S'
  text_cleaning_re = "@\S+|https?:\S+http?:\s|[^A-Za-z0-9]+"


In [30]:
df['Outcome'].value_counts()

Outcome
positive    117670
neutral      59749
negative     22544
Name: count, dtype: int64

In [31]:
from sklearn.utils import resample
df_neg = df[df['Outcome']=='negative']
df_pos = df[df['Outcome']=='positive']
df_neu = df[df['Outcome']=='neutral']
df_neg_sam = resample(df_neg, n_samples=117670)
df_neu_sam = resample(df_neu, n_samples=117670)
df = pd.concat([df_pos,df_neu_sam,df_neg_sam])
df = df.sample(frac=1)
df['Outcome'].value_counts()
##Balancing the values before we start classification
            

Outcome
positive    117670
negative    117670
neutral     117670
Name: count, dtype: int64

In [32]:
from sklearn.pipeline import Pipeline
clf1 = Pipeline([('Vectorizer',CountVectorizer()),('TRansformer',TfidfTransformer()),('Model',MultinomialNB())])
clf2 = Pipeline([('Vectorizer',CountVectorizer()),('TRansformer',TfidfTransformer()),('Model',BernoulliNB())])
clf3 = Pipeline([('Vectorizer',CountVectorizer()),('TRansformer',TfidfTransformer()),('Model',MLPClassifier
(hidden_layer_sizes=(10,10)))])

In [33]:
x = df.content
y = df['Outcome']
xtrain, xtest, ytrain,ytest = train_test_split(x,y, test_size=0.2)
clf1.fit(xtrain,ytrain)

Pipeline(steps=[('Vectorizer', CountVectorizer()),
                ('TRansformer', TfidfTransformer()),
                ('Model', MultinomialNB())])

In [34]:
pred1 = clf1.predict(xtest)
from sklearn .metrics import classification_report,confusion_matrix,precision_score, recall_score,accuracy_score,f1_score
print(classification_report(ytest,pred1))


              precision    recall  f1-score   support

    negative       0.70      0.93      0.80     23479
     neutral       0.93      0.54      0.68     23692
    positive       0.74      0.82      0.78     23431

    accuracy                           0.76     70602
   macro avg       0.79      0.76      0.75     70602
weighted avg       0.79      0.76      0.75     70602



In [35]:
x = df.content
y = df['Outcome']
xtrain, xtest, ytrain,ytest = train_test_split(x,y, test_size=0.2)
clf2.fit(xtrain,ytrain)

Pipeline(steps=[('Vectorizer', CountVectorizer()),
                ('TRansformer', TfidfTransformer()), ('Model', BernoulliNB())])

In [36]:
pred2 = clf2.predict(xtest)
from sklearn .metrics import classification_report,confusion_matrix,precision_score, recall_score,accuracy_score,f1_score
print(classification_report(ytest,pred2))


              precision    recall  f1-score   support

    negative       0.76      0.65      0.70     23601
     neutral       0.91      0.84      0.87     23580
    positive       0.71      0.86      0.78     23421

    accuracy                           0.79     70602
   macro avg       0.79      0.79      0.79     70602
weighted avg       0.79      0.79      0.79     70602



In [56]:

from sklearn.linear_model import LogisticRegression
pred1_tuning = LogisticRegression()
param_grid = {'penalty': ['l1', 'l2', 'elasticnet'],
             'C': [0.1,1,10,100],
             'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag','saga'],
             'max_iter': [100,200,500],
             'multi_class': ['auto', 'ovr', 'multinomial']}
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator=pred1,param_grid=param_grid,cv=5, verbose=1, n_jobs=1)
clf1.fit(xtrain,ytrain)
print(classification_report(ytest,pred1))


              precision    recall  f1-score   support

    negative       0.33      0.44      0.38     23523
     neutral       0.33      0.19      0.24     23526
    positive       0.33      0.36      0.35     23553

    accuracy                           0.33     70602
   macro avg       0.33      0.33      0.32     70602
weighted avg       0.33      0.33      0.32     70602

